In [1]:
import sqlite3
import numpy as np
import string
import random
import os
import datetime

In [2]:
def make_connection():
    db_name = "epidemic.db"
    try:
        con = sqlite3.connect(db_name)
        con.row_factory = sqlite3.Row
        cursor = con.cursor()
    except Exception as e:
        print(e)
        print(f"Error connecting to the database")
        return False, False
    return con, cursor

In [3]:
def randomNumber(be, en):
    return np.random.randint(be, en)

In [4]:
def randomString(size, chars = string.ascii_lowercase):
    return ''.join(random.choice(chars) for x in range(size))

In [5]:
def updateStats(values, con, cursor, stats):
    """
    diseaseid, ncases, ndeaths, spreadprob 
    """
#     print('heyyy')
    diseaseid = values[1]
    spreadprob = random.random()
    new_values = [diseaseid, 1, values[2], spreadprob]
    
    query = f"""
        SELECT * from Stats
        WHERE diseaseid = {diseaseid};
            """
    try:
        cursor.execute(query)
        print('first query')
    except Exception as e:
        return_val = str(e) + "Error in executing query for updating stats"
    data = cursor.fetchone()
    print('frist query done')
    print(data)
    if not data:
        query = """
                INSERT INTO Stats
                VALUES (?, ?, ?, ?);
                """
        stats[diseaseid] = new_values[1:]
        print('new query\n\n')
    else:
        query = f"""
                UPDATE Stats
                SET ncases = ncases + 1, ndeaths = ndeaths + {values[2]}
                WHERE diseaseid = {diseaseid};
                """
        print('Old update\n')
        new_values = False
        stats.update({diseaseid: [stats[diseaseid][0]+1, stats[diseaseid][1] + values[2], spreadprob]})
    print(f"Doing query {query} and values {new_values}")
    print("\n\n\n\n\n\nREAD THIS")
    if new_values:
        print("one")
        cursor.execute(query, new_values)
    else:
        print("two")
        cursor.execute(query)
    try:
        con.commit()
    except Exception as e:
        return_val = str(e) + "Can't execute the query."
    print(f"thththththththht: {return_val}")    
    return return_val
        

In [6]:
def populateDB(values, type = None, stats = None):
    con, cursor = make_connection()
    return_val = "Done"
    
    if type == 'Hospital':
        query = f"""
                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Diseases':
        query = f"""
                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Drugs':
        query = f"""
                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        """
    elif type == 'Pharmacy':
        query = f"""
                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        """
    elif type == 'Cases':
        query = f"""
                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        """
        try:
            print('in try')
            cursor.execute(query, values)
            con.commit()
            print('executed')
#             updateStats(values, con, cursor, stats)
            print('stats updated\n')
        except:
            print("comeon")
            con.close()
    print(query)
    try:
        cursor.execute(query, values)
        con.commit()
        print(type + " query committed")
    except Exception as e:
        return_val = str(e)
    finally:
        con.close()
    print(return_val)
    return return_val

In [7]:
def generateLocation(lat, lon, be, en):
    hosp_loc = []
    for i in range(len(lat)):
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] + ranNum
            lati = lat[i] + ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] + ranNum
            lati = lat[i] - ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] - ranNum
            lati = lat[i] + ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
        for j in range(10):
            ranNum = random.uniform(be, en)
            longi = lon[i] - ranNum
            lati = lat[i] - ranNum
            loc = str(lati) + ' ' + str(longi)
            hosp_loc.append(loc)
    return hosp_loc

In [8]:
dummyLat = [30.553983, 26.619224, 22.463071, 25.712074, 23.957391, 21.116530, 17.927783, 13.145015, 26.146809]
dummyLong = [77.080078, 75.761719, 76.333008, 79.672852, 83.759766, 79.848633, 77.124023, 77.387695, 92.988281]
be = 0.0001
en = 1.5
locs = generateLocation(dummyLat, dummyLong, be, en)

In [9]:
len(locs)

360

# Populating Hospital

In [10]:
be = 10000
en = 99999

hosp_id = []
hosp_userid = []
hosp_name = []
hosp_loc = []

dummyLat = [30.553983, 26.619224, 22.463071, 25.712074, 23.957391, 21.116530, 17.927783, 13.145015, 26.146809]
dummyLong = [77.080078, 75.761719, 76.333008, 79.672852, 83.759766, 79.848633, 77.124023, 77.387695, 92.988281]
be2 = 0.0001
en2 = 1.5

hosp_loc = generateLocation(dummyLat, dummyLong, be2, en2)

for i in range(len(hosp_loc)):
    ID = randomNumber(be, en)
    print(ID)
    Userid = randomNumber(be, en)
    Name = randomString(randomNumber(6, 12))
    
    hosp_id.append(ID)
    hosp_userid.append(Userid)
    hosp_name.append(Name)
    
    a = populateDB(tuple([ID, Name, hosp_loc[i], Userid]), 'Hospital')

61733

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
66023

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
39736

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
75387

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
74846

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
93368

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
53124

                    INSERT INTO Hospital(id, name, location, us

Hospital query committed
Done
56511

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
51351

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
88112

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
11223

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
97850

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
65626

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
64766

                    INSERT INTO H


                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
20723

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
19886

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
69412

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
37611

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
21773

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
80801

                    INSERT INTO Hospital(id, name, location, userid)


Hospital query committed
Done
79747

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
41863

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
55456

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
69166

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
50841

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
98724

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
28792

                    INSERT INTO H

Hospital query committed
Done
89071

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
13057

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
78316

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
69686

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
85564

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
73655

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
89230

                    INSERT INTO H


                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
87952

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
15552

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
18810

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
21644

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
73019

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
60432

                    INSERT INTO Hospital(id, name, location, userid)



                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
33085

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
90925

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
64255

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
67549

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
68528

                    INSERT INTO Hospital(id, name, location, userid)
                    VALUES(?, ?, ?, ?);
        
Hospital query committed
Done
72261

                    INSERT INTO Hospital(id, name, location, userid)


# Populating Diseases

In [11]:
# disease_id = []
# disease_name = []
disease_mos = []
disease_sym = []

disease_name = ['AIDS', 'Maleria', 'Cholera', 'SmallPox', 'Tuberculosis', 'Influenza', 'Dengue', 'Chikungunya', 'Ebola', 'Meningitis']
disease_id = [1,2,3,4,5,6,7,8,9,10]

for i in range(len(disease_id)):
#     id = randomNumber(be, en)
#     name = randomString(randomNumber(6, 10))
    mos = random.choice(['air', 'water', 'food', 'animal'])
    sym = randomString(100)
    
#     disease_id.append(id)
#     disease_name.append(name)
    disease_mos.append(mos)
    disease_sym.append(sym)
    print(i, disease_id[i])
    a = (populateDB(tuple([disease_id[i], disease_name[i], mos, sym]), 'Diseases'))

0 1

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
1 2

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
2 3

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
3 4

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
4 5

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
5 6

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
6 7

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done
7 8

                    INSERT INTO Diseases
                    VALUES(?, ?, ?, ?);
        
Diseases query committed
Done


# Populating Drugs

In [12]:
drug_id = []
drug_name = []
drug_req = []
# drug_avail =[]
disease_drug = {}

be2 = 100
en2 = 999

for i in range(len(disease_id)):
    curDisease = disease_id[i]
    for j in range(randomNumber(1,4)):
        id = randomNumber(be2, en2)
        name = randomString(randomNumber(6, 12))
        req = randomNumber(6, 15)
        avail = randomNumber(1500, 3000)
        
        drug_id.append(id)
        drug_name.append(name)
        drug_req.append(req)
#         drug_avail.append(avail)
        
        #storing drug ids corresponding to each disease
        if(curDisease in disease_drug):
            disease_drug[curDisease].append(id)
        else:
            disease_drug[curDisease] = [id]
        print(tuple([id, name, req, disease_id[i]]))
        a = populateDB(tuple([id, name, req, disease_id[i]]), 'Drugs')

(733, 'mvyesjigpxa', 6, 1)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(373, 'juixwrzthdt', 8, 2)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(425, 'azszon', 14, 2)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(122, 'fstgwltqxf', 14, 3)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(165, 'ryufuzdbh', 12, 3)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(495, 'rmpdbim', 13, 4)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(192, 'bdkvblgvlnv', 6, 4)

                    INSERT INTO Drugs
                    VALUES(?, ?, ?, ?);
        
Drugs query committed
Done
(197, 'wuprwp'

In [13]:
# !cp backup.db epidemic.db

In [14]:
# !ls

# Populating Cases

In [17]:
# id- auto-incremental
case_date = []
case_diseaseid = []
case_death = []
case_location = []

#stats
stats = {}
#for each disease id in stats, we will store ncases, ndeaths and spreadprob

curDate = str(datetime.datetime.now().date())
for i in range(len(disease_id)):
    curDisease = disease_id[i]
    for j in range(randomNumber(50, 100)):
        date = str(randomNumber(2018, 2020)) + '-' 
        tens = str(randomNumber(1, 12))
        if(tens <= str(9)):
            tens = '0' + tens
        
        date += tens + '-' + str(randomNumber(1, 30))
        print(date)
        if(date < curDate):
            death = randomNumber(0, 2)
            location = random.choice(hosp_loc)

            case_date.append(date)
            case_diseaseid.append(curDisease)
            case_death.append(death)
            case_location.append(location)
            a = populateDB(tuple([date, curDisease, death, location]), 'Cases', stats)

2018-05-16
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-04-29
2019-010-14
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-011-29
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-010-7
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-14
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-07-8
2018-010-5
in try
exec

Cases query committed
Done
2018-08-25
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-01-16
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-8
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-01-22
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-02-15
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-08-19
2019-010

Cases query committed
Done
2019-04-20
2018-08-19
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-011-12
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-01-28
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-08-15
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-01-10
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-0

Cases query committed
Done
2019-011-29
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-09-19
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-04-11
2018-09-6
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-04-16
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-08-4
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-010

Cases query committed
Done
2019-07-5
2018-07-3
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-01-18
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-09-2
2019-04-7
2019-08-10
2019-04-20
2019-02-20
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-06-25
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-04-16
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
   

...

Cases query committed
Done
2018-04-5
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-05-29
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-17
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-29
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-05-3
2018-01-16
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-03-2

executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-02-1
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-02-27
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-011-17
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-03-27
2018-07-18
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-1
in try
executed
stats updated


          

Cases query committed
Done
2018-04-17
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-27
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-05-7
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-01-5
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-09-6
2019-08-16
2019-07-24
2019-02-25
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query com

Cases query committed
Done
2019-07-8
2018-04-13
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-07-19
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-04-17
2018-07-25
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2018-08-5
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-01-15
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Don

Cases query committed
Done
2018-05-6
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-08-9
2018-06-18
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-05-7
2019-06-2
2018-03-3
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-07-29
2019-07-6
2018-04-26
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?, ?);
        
Cases query committed
Done
2019-07-4
2018-010-27
in try
executed
stats updated


                    INSERT INTO Cases(date, diseaseid, death, location)
                    VALUES(?, ?, ?

In [16]:
# for i, idx in enumerate(disease_id):
#     print(idx)

In [18]:
# updating stats
for idx, dId in enumerate(disease_id):
    con, cursor = make_connection()
    return_val = "Done"
    query = f"""
            SELECT * FROM Cases
            WHERE diseaseid = {dId}
            """
    cursor.execute(query)
    con.commit()
    
    rows = cursor.fetchall()
    data = [dict(row) for row in rows]
    con.close()
    
#     print(data)
    ncases = len(data)
    print(ncases)
    ndeaths = randomNumber(0, ncases)
    spreadprob = random.random()
    
    con, cursor = make_connection()
    query = f"""
            INSERT INTO Stats
            VALUES(?, ?, ?, ?)
            """
    values = [dId, ncases, ndeaths, spreadprob]
    cursor.execute(query, values)
    con.commit()
    con.close()

114
118
72
84
96
144
128
82
60
58


# Populating Pharmacy

In [19]:
def generate_num_array(size, be, en):
    
    numArray = " ".join([str(randomNumber(be, en)) for i in range(size)])
    return numArray

In [19]:
# s = generate_num_array(5, 1, 10)
# s

In [ ]:
pharm_id = []
pharm_name = []
pharm_loc = []
pharm_drug_id = []
pharm_drug_avail = []

dummyLat = [33.759740, 29.553983, 27.619224, 22.763071, 25.112074, 24.157391, 21.816530, 17.027783, 13.745015, 26.146809]
dummyLong = [76.677269, 77.080078, 75.761719, 76.333008, 79.672852, 83.759766, 79.848633, 78.124023, 77.387695, 92.988281]
beloc = 0.0001
enloc = 1.5

pharm_loc = generateLocation(dummyLat, dummyLong, beloc, enloc)

for i in range(len(pharm_loc)):
    ID = randomNumber(be, en)
    Name = randomString(randomNumber(6, 12))
    ndrugs = randomNumber(10, 30)
    drug_id = generate_num_array(ndrugs, be2, en2)
    drug_avail = generate_num_array(ndrugs, 10, 200)
    
    pharm_id.append(ID)
    pharm_name.append(Name)
    pharm_drug_id.append(drug_id)
    pharm_drug_avail.append(drug_avail)
    
    a = populateDB(tuple([ID, Name, pharm_loc[i], drug_id, drug_avail]), 'Pharmacy')


                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       

Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy


                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

       

Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy

Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy

Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy query committed
Done

                    INSERT INTO Pharmacy
                    VALUES(?, ?, ?, ?, ?);
        
Pharmacy